In [ ]:
import numpy
import scipy.special
from tqdm import tqdm

class NeuralNetwork(object):

    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learning_rate

        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

    def save(self, filename='model.bin'):
        import pickle
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    @classmethod
    def loads(cls, filename='model.bin', force=False):
        if force:
            return cls._loads_model(filename)
        import os
        if not os.path.exists(filename):
            return cls._loads_minst(filename)
        modeltime = os.path.getmtime(filename)
        booktime = os.path.getmtime(os.path.join(os.getcwd(), 'minst.ipynb'))
        if booktime > modeltime:
            return cls._loads_minst(filename)
        return cls._loads_model(filename)

    @classmethod
    def _loads_model(cls, filename='model.bin'):
        import pickle
        try:
            with open(filename, 'rb') as file:
                return pickle.load(file)
        except Exception as e:
            print(e)
            return

    @classmethod
    def _loads_minst(cls, filename='model.bin'):
        import torch
        import torchvision
        from torchvision import transforms, datasets

        root = "../mnist"
        train_data = datasets.MNIST(
            root=root,
            train=True,
            download=True,
            transform=transforms.Compose([transforms.ToTensor()])
            ) 

        test_data = datasets.MNIST(
            root=root,
            train=False,
            download=True,
            transform=transforms.Compose([transforms.ToTensor()])
            ) 

        trainset = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
        testset = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

        input_nodes = 28 * 28 # image size
        hidden_nodes = 100
        output_nodes = 10

        learning_rate = 0.35

        n = cls(input_nodes, hidden_nodes, output_nodes, learning_rate)
        n.train_dataset(trainset)
        n.test_dataset(testset)
        n.save(filename)

    def activate_function(self, inputs):
        return scipy.special.expit(inputs)

    def inverse_activation_function(self, outputs):
        return scipy.special.logit(outputs)

    def train(self, inputs, targets):
        inputs = numpy.array(inputs, ndmin=2).T
        targets = numpy.array(targets, ndmin=2).T

        hidden_inputs = numpy.dot(self.wih, inputs)

        hidden_outputs = self.activate_function(hidden_inputs)

        final_inputs = numpy.dot(self.who, hidden_outputs)

        final_outputs = self.activate_function(final_inputs)

        output_errors = targets - final_outputs

        hidden_errors = numpy.dot(self.who.T, output_errors)

        self.who += self.lr * numpy.dot(
            (output_errors * final_outputs * (1.0 - final_outputs)), 
            numpy.transpose(hidden_outputs)
        )
        
        self.wih += self.lr * numpy.dot(
            (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), 
            numpy.transpose(inputs)
        )

    def train_dataset(self, dataset):
        for data in tqdm(dataset):
            inputs, results = data

            image = inputs[0][0]
            result = results[0]
            inputs = image.reshape(len(image) ** 2)
            
            targets = numpy.zeros(self.onodes) + 0.01
            targets[int(result)] = 0.99
            self.train(inputs, targets)

    def test_dataset(self, dataset):
        total = len(dataset)
        count = 0

        for data in tqdm(dataset):
            inputs, results = data

            image = inputs[0][0]
            result = int(results)
            inputs = image.reshape(len(image) ** 2)
            outputs = self.query(inputs)
            if outputs == result:
                count += 1

        print('\n', count, total, count / total)


    def query(self, inputs):
        inputs = numpy.array(inputs, ndmin=2).T

        hidden_inputs = numpy.dot(self.wih, inputs)

        hidden_outputs = self.activate_function(hidden_inputs)

        final_inputs = numpy.dot(self.who, hidden_outputs)

        final_outputs = self.activate_function(final_inputs)

        outputs = {
            index: var[0] for index, var in enumerate(final_outputs)
        }
        outputs = sorted(outputs.items(), key=lambda e: e[1], reverse=True)[0][0]
        return outputs

    def backquery(self, targets):
        # transpose the targets list to a vertical array
        final_outputs = numpy.array(targets, ndmin=2).T
        
        # calculate the signal into the final output layer
        final_inputs = self.inverse_activation_function(final_outputs)

        # calculate the signal out of the hidden layer
        hidden_outputs = numpy.dot(self.who.T, final_inputs)
        # scale them back to 0.01 to .99
        hidden_outputs -= numpy.min(hidden_outputs)
        hidden_outputs /= numpy.max(hidden_outputs)
        hidden_outputs *= 0.98
        hidden_outputs += 0.01
        
        # calculate the signal into the hidden layer
        hidden_inputs = self.inverse_activation_function(hidden_outputs)
        
        # calculate the signal out of the input layer
        inputs = numpy.dot(self.wih.T, hidden_inputs)
        # scale them back to 0.01 to .99
        inputs -= numpy.min(inputs)
        inputs /= numpy.max(inputs)
        inputs *= 0.98
        inputs += 0.01
        
        return inputs


In [ ]:
n = NeuralNetwork.loads(force=True)

In [ ]:
import os
from PIL import Image
import PIL.ImageOps
import matplotlib.image
import matplotlib.pyplot as plt

for path, dirs, files in  os.walk('./images'):
    for basename in files:
        result = int(basename.split('.')[0])
        filename = os.path.abspath(os.path.join(path, basename))
        image = Image.open(filename)
        image = image.resize((28, 28))
        image = image.convert(mode="L")
        image = PIL.ImageOps.invert(image)
        
       
        inputs = numpy.array(image) / 255
        inputs = inputs.reshape(28 * 28)
        inputs[inputs < 0.1] = 0

        # plt.imshow(inputs.reshape((28, 28)))
        output = n.query(inputs)
        print(result, output)



In [ ]:
targets = numpy.zeros(n.onodes) + 0.01
targets[0] = 0.99

inputs = n.backquery(targets)
inputs = inputs.reshape((28, 28))
plt.imshow(inputs) 
